In [116]:
import os.path
import re
import sys
import numpy as np
import json
import time
import pandas as pd
import h5py
from tqdm import tnrange, tqdm_notebook
import urllib
import glob
import collections
from pattern.en import pluralize, singularize


from random import shuffle, seed
import string
# non-standard dependencies:
import re

import nltk
#nltk.download()

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extract tags from captions for each image in the BM dataset
This notebook extracts key words in each caption of an image to generate tags for images. These tags are used to train a CNN to recognize images automatically.

In [4]:
df = pd.read_hdf('/data/captioning/bm_data.h5', 'df')
df

,print.value,url.value,captions,materials,dates,id
0,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,"From the animal legend; antelope to left, anot...",Woodcut,1912,0
1,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,Woodblock print. Giant lantern in the Kaminari...,n/a,n/a,1
2,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,Woodblock print. Spring night at Ginza.,n/a,n/a,2
3,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,Le Mont de Neuville; street winding to right b...,"Charcoal (?), with watercolour, over monotype",1899,3
4,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,Monochrome woodblock print. Subodai standing. ...,n/a,n/a,4
5,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,Colour woodblock print with powdered mica back...,n/a,n/a,5
6,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,Woodblock print. Beauty with unrolled obi?,n/a,n/a,6
7,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,Colour woodblock print with gold leaf and powd...,n/a,n/a,7
8,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,"Illustrated book, orihon , 5 vols . Instructio...",n/a,n/a,8
9,http://collection.britishmuseum.org/id/object/...,http://www.britishmuseum.org/collectionimages/...,"Illustrated book, folding album, 2 vols. Varie...",n/a,n/a,9


In [ ]:
#imgs = json.load(open("/data/captioning/json_img_gs.json", 'r'))
#seed(123) # make reproducible
#shuffle(imgs) # shuffle the order

In [91]:
imgs[1]

{u'captions': [u'Tamar, plate 6: view of a park, with two large trees in the centre foreground to the right, herd of deer resting below, line of further trees behind to the right, clearing to the left, gently sloping down to another forest below a hill in the left background; early proof, touched'],
 u'file_path': u'217914.jpg_postproc.jpg',
 'processed_tokens': [[u'tamar',
   u'plate',
   u'6',
   u'view',
   u'of',
   u'a',
   u'park',
   u'with',
   u'two',
   u'large',
   u'trees',
   u'in',
   u'the',
   u'centre',
   u'foreground',
   u'to',
   u'the',
   u'right',
   u'herd',
   u'of',
   u'deer',
   u'resting',
   u'below',
   u'line',
   u'of',
   u'further',
   u'trees',
   u'behind',
   u'to',
   u'the',
   u'right',
   u'clearing',
   u'to',
   u'the',
   u'left',
   u'gently',
   u'sloping',
   u'down',
   u'to',
   u'another',
   u'forest',
   u'below',
   u'a',
   u'hill',
   u'in',
   u'the',
   u'left',
   u'background',
   u'early',
   u'proof',
   u'touched']]}

In [59]:
# Adopted from neuraltalk2
import unicodedata
import sys

tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P'))
def remove_punctuation_2(text):
    return text.translate(tbl)


import enchant
d = enchant.Dict("en_US")
counter = 0
for w in vocab:
    if not d.check(w) or w in stopwords.words('english'):
        counter+=1
        
print len(vocab)
print counter

def prepro_captions(imgs):
  
  # preprocess all the captions
  print 'example processed tokens:'
  for i,img in enumerate(imgs):
    img['processed_tokens'] = []
    for j,s in enumerate(img['captions']):
      txt = remove_punctuation_2(s)
      txt = txt.lower().strip().split()
    img['processed_tokens'].append(txt)
    if i < 10 and j == 0: print txt



In [62]:
d.check('232kfs352')

False

In [69]:

count_thr = 10

# tokenization and preprocessing
prepro_captions(imgs)

# count up the number of words
counts = {}
for img in imgs:
    for txt in img['processed_tokens']:
      for w in txt:
        counts[w] = counts.get(w, 0) + 1
cw = sorted([(count,w) for w,count in counts.iteritems()], reverse=True)
print 'top words and their counts:'
print '\n'.join(map(str,cw[:20]))

# print some stats
total_words = sum(counts.itervalues())
print 'total words:', total_words
bad_words = [w for w,n in counts.iteritems() if n <= count_thr]
vocab = [w for w,n in counts.iteritems() if n > count_thr]
bad_count = sum(counts[w] for w in bad_words)
print 'number of bad words: %d/%d = %.2f%%' % (len(bad_words), len(counts), len(bad_words)*100.0/len(counts))
print 'number of words in vocab would be %d' % (len(vocab), )
print 'number of UNKs: %d/%d = %.2f%%' % (bad_count, total_words, bad_count*100.0/total_words)

# lets look at the distribution of lengths as well
#   sent_lengths = {}
#   for img in imgs:
#     for txt in img['processed_tokens']:
#       nw = len(txt)
#       sent_lengths[nw] = sent_lengths.get(nw, 0) + 1
#   max_len = max(sent_lengths.keys())
#   print 'max length sentence in raw data: ', max_len
#   print 'sentence length distribution (count, number of words):'
#   sum_len = sum(sent_lengths.values())
#   for i in xrange(max_len+1):
#     print '%2d: %10d   %f%%' % (i, sent_lengths.get(i,0), sent_lengths.get(i,0)*100.0/sum_len)

# lets now produce the final annotations
# if bad_count > 0:
# # additional special UNK token we will use below to map infrequent words to
# print 'inserting the special UNK token'
# vocab.append('UNK')

# for img in imgs:
#     img['final_captions'] = []
#     for txt in img['processed_tokens']:
#       caption = [w if counts.get(w,0) > count_thr else 'UNK' for w in txt]
#       img['final_captions'].append(caption)

# create the vocab
#vocab = build_vocab(imgs, params)
#itow = {i+1:w for i,w in enumerate(vocab)} # a 1-indexed vocab translation table
#wtoi = {w:i+1 for i,w in enumerate(vocab)} # inverse table


example processed tokens:
[u'portrait', u'of', u'jeronimo', u'de', u'bran', u'threequarter', u'length', u'to', u'left', u'holding', u'a', u'baton', u'after', u'jan', u'lievens']
[u'tamar', u'plate', u'6', u'view', u'of', u'a', u'park', u'with', u'two', u'large', u'trees', u'in', u'the', u'centre', u'foreground', u'to', u'the', u'right', u'herd', u'of', u'deer', u'resting', u'below', u'line', u'of', u'further', u'trees', u'behind', u'to', u'the', u'right', u'clearing', u'to', u'the', u'left', u'gently', u'sloping', u'down', u'to', u'another', u'forest', u'below', u'a', u'hill', u'in', u'the', u'left', u'background', u'early', u'proof', u'touched']
[u'portrait', u'as', u'a', u'child', u'half', u'length', u'in', u'an', u'oval', u'long', u'hair', u'wearing', u'neckcloth', u'coat', u'sash', u'and', u'badge', u'with', u'star', u'c16991700']
[u'plate', u'3', u'summer', u'landscape', u'with', u'a', u'rider', u'on', u'a', u'rearing', u'horse', u'at', u'right', u'and', u'beyond', u'a', u'cart', 

# BELOW IS REAL CODE

1. Use NLTK tagger to get the noun words
2. Get top 500 words
3. for each words, get a list of indices of imgs with that tag

4. store this information into json file

In [101]:

pos = {}
target_pos = ['NN', 'NNP', 'NNS', 'NNPS', 'FW']
for i,img in enumerate(imgs):
    img['processed_tokens'] = []
    sents = nltk.sent_tokenize(img['captions'][0])
    for sent in sents:
        tokens = nltk.word_tokenize(sent)
        pos_document = nltk.pos_tag(tokens)
        for pos_word in pos_document:
            word, part = pos_word
            if part in target_pos:
                img['processed_tokens'].append(word)
                if part not in pos:
                    pos[part] = collections.Counter()
                pos[part][word]+=1
    if (i % 10000 == 0):
        print i

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
{'NNPS': Counter({u'Hans': 574, u'Commons': 285, u'Frans': 265, u'Cries': 171, u'Lords': 118, u'Rights': 101, u'Romans': 93, u'Albans': 89, u'Israelites': 84, u'Provinces': 81, u'Characters': 80, u'Elders': 77, u'Cities': 75, u'Gardens': 74, u'Indians': 67, u'Maries': 67, u'Gentlemans': 58, u'Belgians': 55, u'Graces': 55, u'Teniers': 52, u'Cotes': 51, u'Christians': 44, u'Brothers': 44, u'Orl\xe9ans': 44, u'Jews': 43, u'Americans': 43, u'Evans': 42, u'Thames': 41, u'States': 41, u'Russians': 38, u'Protestants': 36, u'Antiquities': 35, u'Philistines': 35, u'Catholics': 34, u'Friends': 33, u'Snyders': 33, u'Dutch': 30, u'Egyptians': 30, u'Sciences': 30, u'Assyrians': 29, u'Trojans': 29, u'Rubens': 28, u'Angels': 28, u'Poems': 28, u'Europeans': 26, u'Apostles': 26, u'Palaces': 26, u'Days': 26, u'Carthaginians': 26, u'Nether

In [157]:
# with open('/data/captioning/img_gs_with_parsed_captions.json', 'w') as outfile:
#     json.dump(imgs, outfile)

#imgs = json.load(open("/data/captioning/img_gs_with_parsed_captions.json", 'r'))

# with open('/data/captioning/img_gs_nltk_pos_tags.json', 'w') as outfile:
#     json.dump(pos, outfile)

#pos = json.load(open("/data/captioning/img_gs_nltk_pos_tags.json", 'r'))


In [138]:
pos['NN'].most_common(500)


[(u'left', 62203),
 (u'right', 60980),
 (u'man', 39644),
 (u'background', 34789),
 (u'hand', 34668),
 (u'foreground', 34474),
 (u'illustration', 30150),
 (u'woman', 29903),
 (u'head', 26193),
 (u'state', 25368),
 (u'Portrait', 24952),
 (u'centre', 22892),
 (u'front', 18848),
 (u'proof', 17781),
 (u'profile', 16097),
 (u'View', 15650),
 (u'landscape', 14365),
 (u'length', 13944),
 (u'oval', 13498),
 (u'figure', 13455),
 (u'table', 12966),
 (u'coat', 12131),
 (u'Plate', 11919),
 (u'view', 11621),
 (u'half-length', 11079),
 (u'book', 11039),
 (u'hat', 10979),
 (u'distance', 10792),
 (u'frame', 10682),
 (u'river', 10678),
 (u'series', 10676),
 (u'hair', 10617),
 (u'side', 10571),
 (u'ground', 10490),
 (u'card', 10438),
 (u'arm', 9884),
 (u'tree', 9459),
 (u'viewer', 9365),
 (u'plate', 9253),
 (u'dress', 8654),
 (u'title', 8449),
 (u'dog', 8085),
 (u'print', 7867),
 (u'scene', 7768),
 (u'standing', 7451),
 (u'child', 7229),
 (u'wig', 7133),
 (u'wall', 7125),
 (u'group', 6891),
 (u'sword', 6

In [158]:
# add the plural forms to the singular word counts(but keep the plural counts)
import copy
pos['NN+NNS'] = copy.deepcopy(pos['NN'])

In [159]:
print len(pos['NN+NNS'])
print len(pos['NNS'])

40589
15654


In [160]:
c = 0
for w, count in pos['NNS'].items():
    ws = singularize(w) 
    if ws in pos['NN+NNS']:
        c+=1
        pos['NN+NNS'][ws] += count
print c

8542


In [1]:
pos['NN+NNS'].most_common(100)

NameError: name 'pos' is not defined

In [141]:
pos['NNP'].most_common(100)

[(u'St', 17835),
 (u'Portrait', 11695),
 (u'John', 10525),
 (u'DRAFT', 10123),
 (u'Christ', 10109),
 (u'Trade', 9820),
 (u'Virgin', 8127),
 (u'Plate', 5683),
 (u'William', 5586),
 (u'London', 5225),
 (u'Woodblock', 4866),
 (u'King', 4476),
 (u'Child', 4187),
 (u'George', 4081),
 (u'Peter', 3874),
 (u'Paul', 3741),
 (u'Charles', 3711),
 (u'Mary', 3598),
 (u'Joseph', 3538),
 (u'II', 3476),
 (u'James', 3389),
 (u'Duke', 3262),
 (u'Thomas', 3129),
 (u'Lord', 3104),
 (u'Reynolds', 2828),
 (u'Holy', 2702),
 (u'III', 2676),
 (u'Prince', 2631),
 (u'Henry', 2472),
 (u'Sir', 2397),
 (u'House', 2390),
 (u'Fox', 2352),
 (u'England', 2335),
 (u'Cupid', 2143),
 (u'Queen', 2135),
 (u'Turner', 2004),
 (u'Interior', 2000),
 (u'Napoleon', 1978),
 (u'Rubens', 1951),
 (u'Baptist', 1907),
 (u'Magazine', 1846),
 (u'IV', 1842),
 (u'Louis', 1806),
 (u'Richard', 1797),
 (u'Roman', 1739),
 (u'Edward', 1726),
 (u'Venus', 1712),
 (u'JMW', 1711),
 (u'David', 1695),
 (u'Street', 1689),
 (u'Art', 1679),
 (u'Great', 

In [161]:
wtoi = {} # each word => list of image ids 

for pos_counter in (pos['NN+NNS'], pos['NNS'], pos['NNP']):
    for w, count in pos_counter.iteritems():
        if count >= 300:
            wtoi[w] = []
len(wtoi)           

1727

In [163]:
post = '.jpg_postproc.jpg'

def id_to_fn(id):
    return str(id) + post
    
def fn_to_id(fn):
    return int(fn.replace(post, ''))
    
for img in imgs:
    wl = img['processed_tokens']
    for w in wl:
        if w in pos['NNS']:
            ws = singularize(w)
            if ws in wtoi:
                wtoi[ws].append(fn_to_id(img['file_path']))
        if w in wtoi:
            wtoi[w].append(fn_to_id(img['file_path']))


In [164]:
[(w, len(imgs)) for w, imgs in wtoi.items()[:50]]

[(u'hats', 992),
 (u'Stuart', 311),
 (u'woods', 421),
 (u'hanging', 740),
 (u'lace', 2914),
 (u'Plate', 17651),
 (u'aquatint', 368),
 (u'swan', 569),
 (u'trousers', 712),
 (u'G.', 404),
 (u'merchant', 363),
 (u'Henri', 765),
 (u'vase', 1932),
 (u'Henry', 2472),
 (u'Old', 1269),
 (u'fireplace', 717),
 (u'monkeys', 331),
 (u'chapel', 1742),
 (u'school', 314),
 (u'parrot', 716),
 (u'wooden', 704),
 (u'Hercules', 1036),
 (u'Paul', 3741),
 (u'Windsor', 423),
 (u'clothes', 1155),
 (u'farmhouse', 363),
 (u'Bavaria', 592),
 (u'force', 342),
 (u'ornate', 917),
 (u'feathers', 1254),
 (u'horn', 1726),
 (u'Matthew', 389),
 (u'street', 8330),
 (u'quill', 796),
 (u'fingers', 451),
 (u'Hamilton', 528),
 (u'net', 532),
 (u'niche', 1078),
 (u'men', 14052),
 (u'herd', 650),
 (u'path', 3449),
 (u'disciples', 327),
 (u'daughter', 1239),
 (u'rests', 382),
 (u'Miss', 531),
 (u'study', 1607),
 (u'Queen', 2226),
 (u'anchor', 667),
 (u'tankard', 575),
 (u'smoke', 1027)]

In [165]:
with open('/data/captioning/bm_tags_to_imgs.json', 'w') as outfile:
    json.dump(wtoi, outfile)

In [156]:
wtoi.keys()

[u'hats',
 u'Stuart',
 u'woods',
 u'hanging',
 u'lace',
 u'Plate',
 u'aquatint',
 u'swan',
 u'trousers',
 u'G.',
 u'merchant',
 u'Henri',
 u'vase',
 u'Henry',
 u'Old',
 u'fireplace',
 u'monkeys',
 u'chapel',
 u'school',
 u'parrot',
 u'wooden',
 u'Hercules',
 u'Paul',
 u'Windsor',
 u'clothes',
 u'farmhouse',
 u'Bavaria',
 u'force',
 u'ornate',
 u'feathers',
 u'horn',
 u'Matthew',
 u'street',
 u'quill',
 u'fingers',
 u'Hamilton',
 u'net',
 u'niche',
 u'men',
 u'herd',
 u'path',
 u'disciples',
 u'daughter',
 u'rests',
 u'Miss',
 u'study',
 u'Queen',
 u'tankard',
 u'smoke',
 u'skull-cap',
 u'prints',
 u'Group',
 u'straw',
 u'Harvey',
 u'St.',
 u'bayonet',
 u'Behind',
 u'cane',
 u'Wales',
 u'landscape',
 u'army',
 u'coins',
 u'June',
 u'arms',
 u'music',
 u'tradesman',
 u'type',
 u'glass',
 u'V',
 u'Kabuki',
 u'hole',
 u'shepherds',
 u'word',
 u'room',
 u'work',
 u'roof',
 u'foliage',
 u'shelve',
 u'elbow',
 u'fortress',
 u'Poem',
 u'Le',
 u'quiver',
 u'serpent',
 u'La',
 u'Antwerp',
 u'dol